In [68]:
# Import Libraries
import pandas as pd
import altair as alt
import csv

# Function to read raw data and populate into Pandas dataframe
def readAndPopulateDF(src, colName):
    attributes = []
    records = []
    with open(src) as inFile:
        reader = csv.reader(inFile)
        for row in reader:
            if (row[0] == colName):
                attributes = row
                continue
            records.append(row)
    return pd.DataFrame(records, columns = attributes)

# Define constant values and 
# NOTE: I tried defining just 1, but the I did not find a solution to pass in a None value for color
PAD = 20
WIDTH = 400
HEIGHT = 250
X_VAL = 'Year:O'
X_TITLE = 'Year'

# Function to get time series visualization with 1 series
def getSingleSeriesGraph(src, yVal, yTitle, chartTitle):
    return alt.Chart(src).mark_line().encode(
        x = alt.X(X_VAL, axis=alt.Axis(title=X_TITLE)),
        y = alt.Y(yVal, axis=alt.Axis(title=yTitle)),
    ).properties(
        title = chartTitle,
        width = WIDTH,
        height = HEIGHT
    ).configure(
        padding={"left": PAD, "top": PAD, "right": PAD, "bottom": PAD}
    )

# Function to get time series visualization with multiplie series
def getMultiSeriesGraph(src, xVal, xTitle, yVal, yTitle, chartTitle, chartColor):
    return alt.Chart(src).mark_line().encode(
        x = alt.X(xVal, axis = alt.Axis(title = xTitle)),
        y = alt.Y(yVal, axis = alt.Axis(title = yTitle)),
        color = chartColor
    ).properties(
        title = chartTitle,
        width = WIDTH,
        height = HEIGHT
    ).configure(
        padding={"left": PAD, "top": PAD, "right": PAD, "bottom": PAD}
    )

In [69]:
df = readAndPopulateDF('fossil-fuel-co2-emissions-by-nation_filtered.csv', 'Year')
df

,Year,Country,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total)
0,1950,AFGHANISTAN,23,6,18,0,0,0,0,0
1,1950,ALBANIA,81,12,68,0,2,0,0.07,0
2,1950,ALGERIA,1033,514,475,0,44,0,0.12,612
3,1950,ANGOLA,51,16,34,0,0,0,0.01,0
4,1950,ARGENTINA,8168,972,6982,0,214,0,0.48,124
...,...,...,...,...,...,...,...,...,...,...
12457,2014,VIET NAM,45517,19246,12694,5349,8229,0,0.49,761
12458,2014,WALLIS AND FUTUNA ISLANDS,6,0,6,0,0,0,0.44,1
12459,2014,YEMEN,6190,137,5090,581,381,0,0.24,153
12460,2014,ZAMBIA,1228,132,797,0,299,0,0.08,33


In [70]:
# NOTE: I wasn't sure if I was supposed to include Field ID 7, 8, 9, 10 all in the same visualization since the numbers 
# don't logically fit in the y-axis of Fuel. I made 2 functions: 1 for multiple and 1 for single series visualization

df_2b = df.copy()

# Get all the unique values of countries and filter out USA
uniqueCountryVals = df_2b.Country.unique().tolist()
uniqueCountryVals.remove('UNITED STATES OF AMERICA')

# For each country, remove all rows with Country attribute of that country value
for val in uniqueCountryVals:
    df_2b.drop(df_2b[df_2b['Country'] == val].index, inplace = True)

# No longer need Country column
del df_2b['Country']

# Create a new dataframe df_2b_transformed with the 3 attributes
attr = ['Emission Source', 'Year', 'Amount']
df_2b_transformed = pd.DataFrame(columns=attr)

# Loop through each entry in df_2b
for index, row in df_2b.iterrows():
    rowTrimmed = row.tolist()[1:5]
    # Loop through each att in the row
    for i in range(len(rowTrimmed)):
        # Append new information
        df_2b_transformed = df_2b_transformed.append({
                    attr[0]: df_2b.columns.tolist()[i+1], 
                    attr[1]: row.tolist()[0],
                    attr[2]: rowTrimmed[i]}, 
        ignore_index=True)

df_2b_transformed

,Emission Source,Year,Amount
0,Total,1950,692124
1,Solid Fuel,1950,343131
2,Liquid Fuel,1950,244760
3,Gas Fuel,1950,87123
4,Total,1951,712983
...,...,...,...
255,Gas Fuel,2013,379659
256,Total,2014,1432855
257,Solid Fuel,2014,450047
258,Liquid Fuel,2014,576531


In [71]:
getMultiSeriesGraph(df_2b_transformed, 
                    'Year:O', 
                    'Year', 
                    'Amount:Q', 
                    'Amount of Emission', 
                    'Emission of USA since 1950', 
                    'Emission Source')

alt.Chart(...)

In [72]:
getSingleSeriesGraph(df_2b, 'Cement:Q', 'Cement Usage', 'Cement Usage of USA since 1950')

alt.Chart(...)

In [73]:
getSingleSeriesGraph(df_2b, 'Gas Flaring:Q', ' Gas Flaring', 'Gas Flaring of USA since 1950')

alt.Chart(...)

In [74]:
getSingleSeriesGraph(df_2b, 'Per Capita:Q', 'Per Capita', 'Per Capita of USA since 1950')

alt.Chart(...)

In [75]:
getSingleSeriesGraph(df_2b, 'Bunker fuels (Not in Total):Q', 'Bunker Fuel Usage of USA since 1950', 'Bunker Fuel Usage')

alt.Chart(...)

In [76]:
df_2c = df.copy()

# Select countries to chose from
countryVals = ['UNITED STATES OF AMERICA', 'CANADA', 
                    'MEXICO', 'UNITED KINGDOM', 
                    'ARGENTINA','CHINA (MAINLAND)', 
                    'JAPAN', 'REPUBLIC OF KOREA', 
                    'INDONESIA', 'INDIA']

# Create a new dataframe df_2c_transformed with the 4 attributes
attr = ['Country', 'Year', 'Per Capita', 'Total']
df_2c_transformed = pd.DataFrame(columns=attr)

# Loop thorugh the df_2c
for index, row in df_2c.iterrows():
    # If the country is included in countryVals, append record with 4 attr to df_2c_transformed
    if row.tolist()[1] in countryVals:
        df_2c_transformed = df_2c_transformed.append({
                     attr[0]: df_2c.iloc[index]['Country'], 
                     attr[1]: df_2c.iloc[index]['Year'],
                     attr[2]: df_2c.iloc[index]['Per Capita'],
                     attr[3]: df_2c.iloc[index]['Total']}, 
        ignore_index=True)

df_2c_transformed

,Country,Year,Per Capita,Total
0,ARGENTINA,1950,0.48,8168
1,CANADA,1950,3.06,42070
2,CHINA (MAINLAND),1950,0.04,21465
3,INDIA,1950,0.05,18178
4,INDONESIA,1950,0.04,2683
...,...,...,...,...
645,JAPAN,2014,2.61,331074
646,MEXICO,2014,1.04,130971
647,REPUBLIC OF KOREA,2014,3.2,160119
648,UNITED KINGDOM,2014,1.78,114486


In [77]:
getMultiSeriesGraph(df_2c_transformed, 
                    'Year:O',
                    'Year',
                    'Per Capita:Q', 
                    'Per Capita', 
                    'Per Capita of 10 Countries since 1950',
                    'Country')

alt.Chart(...)

In [78]:
getMultiSeriesGraph(df_2c_transformed, 
                    'Year:O',
                    'Year',
                    'Total:Q', 
                    'Total CO2 Emission', 
                    'Total CO2 Emission of 10 Countries since 1950',
                    'Country')

alt.Chart(...)

In [79]:
df = readAndPopulateDF('all_stocks_5yr.csv', 'date')
df

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.1,14.27,8126000,AAL
3,2013-02-13,14.3,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [80]:
# NOTE: This cell might take about 20 seconds to run due to the 
# large number of records in the df. Please be patient.

df_2d = df.copy()

# Select stocks to chose from
stockVals = ['AAL','UAL','DAL']

# Create a new dataframe df_2d_transformed with the 4 attributes
attr = ['Name', 'close', 'volume', 'date']
df_2d_transformed = pd.DataFrame(columns=attr)

# Loop thorugh the df_2d
for index, row in df_2d.iterrows():
    if row.tolist()[6] in stockVals:
        df_2d_transformed = df_2d_transformed.append({
                     attr[0]: df_2d.iloc[index]['Name'], 
                     attr[1]: df_2d.iloc[index]['close'],
                     attr[2]: df_2d.iloc[index]['volume'],
                     attr[3]: df_2d.iloc[index]['date']}, 
        ignore_index=True)

df_2d_transformed

,Name,close,volume,date
0,AAL,14.75,8407500,2013-02-08
1,AAL,14.46,8882000,2013-02-11
2,AAL,14.27,8126000,2013-02-12
3,AAL,14.66,10259500,2013-02-13
4,AAL,13.99,31879900,2013-02-14
...,...,...,...,...
3772,UAL,67.15,5603314,2018-02-01
3773,UAL,65.68,4189758,2018-02-02
3774,UAL,63.4,5433829,2018-02-05
3775,UAL,63.95,4670812,2018-02-06


In [81]:
getMultiSeriesGraph(df_2d_transformed, 
                    'date:O',
                    'Date',
                    'close:Q', 
                    'Close Daily Price', 
                    'Closing Stock Price of 3 Stocks in from 2013 to 2018',
                    'Name')

alt.Chart(...)